In [117]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import os
import shutil
import cv2
from tqdm import trange

#import glob2

from scipy.io import loadmat
import scipy
#import hdf5storage
import csv
from tqdm import trange

import glob

Autosaving every 180 seconds


In [185]:
############################################
############ MAKE VIDEO BASES ON TRACES ####
############################################

#
def make_video(video_name,
               fname_chain_ids,
               fname_traces,
               start,
               end):
    
    from tqdm import trange

    # colors are fixed
    colors_4= ['blue','red', 'yellow', 'green', 'yellow']  # Female is actually red, but brg is the color scheme

    original_vid = cv2.VideoCapture(video_name)
    font = cv2.FONT_HERSHEY_PLAIN

    # video sizes
    size_vid = np.array([1280,1024])
    scale = 1
    dot_size1 = 8//scale
    dot_size2 = 4//scale
    shift = 10

    n_features = 14
    n_animals = 4
    
    # 
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

    # Initialize video out
    fname_out = video_name[:-4]+"_locs_added_"+str(start)+"_"+str(end)+".mp4"
    fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
    video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0]//scale,size_vid[1]//scale), True)

    for n in trange(start,end, 1):
        ret, frame = original_vid.read()
        cv2.putText(frame, str(n), (50, 100), font, 5, (255, 255, 0), 5)
        frame = frame[::scale, ::scale]

        # loop over the info
        for k in range(n_animals):

            ###############################################
            ############# CNN ALG RESULTS #################
            ###############################################
            y = data_classification[n, k,:,0]
            x = data_classification[n, k,:,1]

            idx = np.where(np.isnan(x)==False)[0]
            plot_flag = True
            if idx.shape[0]>0:

                xs = x[idx]
                ys = y[idx]

                for p in range(xs.shape[0]):
                    x=int(xs[p])//scale+shift
                    y=int(ys[p])//scale+shift

                    frame[x-dot_size2:x+dot_size2,y-dot_size2:y+dot_size2]= (np.float32(
                        matplotlib.colors.to_rgb(colors_4[k]))*255.).astype('uint8')

                if plot_flag:

                    cv2.putText(frame, str(chain_ids[n,k]), (y, x), font, 2, (255, 255, 0), 5)
                    plot_flag=False


            ###############################################
            ######### HUNGARIAN ALG RESULTS ###############
            ###############################################
            if False:
                y = data_hungarian[n, k,:,0]
                x = data_hungarian[n, k,:,1]

                idx = np.where(np.isnan(x)==False)[0]
                plot_flag = True
                if idx.shape[0]>0:

                    xs = x[idx]
                    ys = y[idx]

                    for p in range(xs.shape[0]):
                        x=int(xs[p])//scale
                        y=int(ys[p])//scale

                        frame[x-dot_size1:x+dot_size1,y-dot_size1:y+dot_size1]= (np.float32(
                                    matplotlib.colors.to_rgb(colors_4[k]))*255.).astype('uint8')



        #print ("")
        video_out.write(frame)

        #print ("")

    video_out.release()
    original_vid.release()
    #cv2.destroyAllWindows()

    
#  
def get_animal_ids(frame,
                  chain_ids,
                  root_dir,
                  vals,
                  classes,
                  n_animals,
                  comments=False):

    # 
    dirs = chain_ids[frame]

    if comments:
        print ("DIRS: ", dirs)
    
    animal_ids = []
    animal_ids2 = []
    threshold_purity1 = 0.5
    threshold_purity2 = 0.7

    # HUNGARIAN ALGORITHM BASED SEARCH FOR MOST CORRECT IDS
    best_ids_cts = np.zeros((5,5),'float32')
    animals3 = []
    for ctr_dir, dir_ in enumerate(dirs):
        #print ("2nd Loop: ", dir_)
        if np.isnan(dir_)==False:
            
            # find the classifier values for that segment
            idx = np.where(vals==dir_)[0]

            # find predicted id;
            # First find 
            fnames = np.sort(glob.glob(root_dir+str(int(dir_))+'/*.npz',recursive = False))

            ctr_chain = 0
            for fname in fnames:
                if ('frame_'+str(frame).zfill(7)) in fname:
                     break 
                ctr_chain+=1
        
            animal_id3 = classes[idx][ctr_chain]
            animals3.append(animal_id3)
            
            idx5 = np.where(classes[idx]<1E10)[0]
            classes_local = classes[idx[idx5]]
            
            classes_unique = np.unique(classes_local, return_counts=True)
            #print ("best id: ", best_id)
            
            best_ids_cts[ctr_dir, classes_unique[0]]=classes_unique[1]/idx.shape[0]
        else:
            animals3.append(None)
    
    # FIRST LOOP TO CHECK ANY GREAT MATCHES AND ZERO OUT EVERYTHING ELSE
    for k1 in range(n_animals):
        temp = best_ids_cts[k1]
        max_val = np.max(temp)
        if max_val>threshold_purity2:
            idx8 = np.argmax(temp)
            best_ids_cts[:,idx8]=0
            best_ids_cts[k1,idx8]=max_val
            
    if comments:
        print (best_ids_cts)
    
    # RENORMALIZE PROBABILITIES AFTER ZEROING OUT
    for k1 in range(n_animals):
        sum_ = best_ids_cts[k1].sum()
        if sum_>0:
            best_ids_cts[k1]*= 1./sum_
    if comments:
        print (best_ids_cts)    
        
    # FINAL LOOP TO GRAB WHATEVER IS LEFT
    for k1 in range(n_animals):
        max_val = np.max(best_ids_cts[k1])
        if max_val==0: # or max_val<threshold_purity1:
            animal_ids.append(None)
            animal_ids2.append(animals3[k1])
            continue
        else:
            idx9 = np.argmax(best_ids_cts[k1])
            #print ("idx9: ", idx9, classes_unique[0])
            animal_ids.append(idx9)
            animal_ids2.append(animals3[k1])
            
            best_ids_cts[:,idx9]= 0
            # must zero out all the others
            
    if comments:
        print (best_ids_cts)   
        
    return animal_ids, animal_ids2, dirs



def get_animal_ids_function2(fname_classification,
                             fname_traces_inferences,
                             chain_ids,
                             root_dir,
                             comments,
                             n_animals,
                             n_features):
    
    fname_out = os.path.split(fname_classification)[0]+"/animal_arrays.npy"
    if os.path.exists(fname_out)==False:

        data = np.load(fname_classification, allow_pickle=True)
        vals = data['vals']
        frames = np.hstack(data['frames_array'])
        print ("# of lableed frames: ", vals.shape)
        classes = data['classes']

        reassembled = np.load(fname_traces_inferences)

        #reassembled = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/pickle/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz')
        tracesx_re = reassembled[:,:,0]
        tracesy_re = reassembled[:,:,1]
        print ("Lenght of tracessx: ", tracesx_re.shape)

        start = 1
        end = tracesx_re.shape[0]

        # INITIALIZE ARRAYS TO BE SAVED
        animal_ids_hungarian = np.zeros((tracesx_re.shape[0], n_networks), 'float32')
        animal_arrays = np.zeros((n_networks, tracesx_re.shape[0], 2), 'float32')
        
        # loop over all video frames
        for n in trange(start,end, 1):
            # animal_ids is the hungarian output;
            # animal_ids is the CNN output
            animal_ids, animal_ids2, dirs = get_animal_ids(n,
                                                        chain_ids,
                                                        root_dir,
                                                        vals,
                                                        classes,
                                                        n_animals,
                                                        comments)

            # 
            for k in range(0, n_features*n_animals,n_features):
                y = tracesx_re[n,k:k+n_features]
                x = tracesy_re[n,k:k+n_features]

                idx = np.where(x!=0)[0]
                x = x[idx]
                y = y[idx]
                if x.shape[0]==0:
                    x_ave = None
                    y_ave = None
                else:
                    x_ave = np.nanmean(x)
                    y_ave = np.nanmean(y)

                if animal_ids[k//n_features] is None:
                    continue
                animal_arrays[animal_ids[k//n_features], n] = x_ave, y_ave

            animal_ids_hungarian[n]=animal_ids 

        np.save(fname_out, animal_arrays) # these are the mean location of the data
        np.save(fname_out[:-4]+"_animal_ids_hungarian.npy", animal_ids_hungarian)
    
    else:
        animal_arrays = np.load(fname_out)
        animal_ids_hungarian = np.load(fname_out[:-4]+"_animal_ids_hungarian.npy")

    print (animal_arrays.shape)
    n_frames = animal_arrays.shape[1]

    # 
    traces_inferences_classification_hungarian = np.zeros((n_frames, 
                                                           n_animals, 
                                                           n_features, 
                                                           2), 'float32')+np.nan
    #
    print ("Combining hungarian and sleap output")
    
    # combine hungarian output and slp output
    traces_inferences = np.load(fname_traces_inferences)
    tracesx_re = traces_inferences[:,:,0]
    tracesy_re = traces_inferences[:,:,1]
    for frame in trange(1,n_frames,1):
        animals_selected = animal_ids_hungarian[frame]
        #print ("animals_selected: ", animals_selected)

        for k in range(n_networks):
            if np.isnan(animals_selected[k]):
                continue
            
            animal_selected = int(animals_selected[k])
                
            featuresx = tracesx_re[frame,k*n_features:(k+1)*n_features]
            featuresy = tracesy_re[frame,k*n_features:(k+1)*n_features]

            # 
            traces_inferences_classification_hungarian[frame,
                          animal_selected,:,0]=featuresx

            traces_inferences_classification_hungarian[frame,
                          animal_selected,:,1]=featuresy

    np.save(fname_traces_inferences[:-4]+"_hungarian.npy", 
            traces_inferences_classification_hungarian)
            
    print ("DONE making animal_arrays: ", animal_arrays.shape)
    
    return animal_arrays, animal_ids_hungarian, traces_inferences_classification_hungarian


def get_single_feature_traces(fname_classification,
                              fname_traces_inferences,
                              animal_ids_hungarian,
                              n_networks,
                              features):
    
    fname_out = os.path.split(fname_classification)[0] + "/locs_array.npy"
    if os.path.exists(fname_out)==False:
        
        reassembled = np.load(fname_traces_inferences)
        tracesx_re = reassembled[:,:,0]
        tracesy_re = reassembled[:,:,1]
        print ("Lenght of tracessx: ", tracesx_re.shape)
        
        locs_array = np.zeros((n_networks, animal_ids_hungarian.shape[0],2),'float32')+np.nan
        
        #for p in ids:
        from tqdm import trange
        for p in trange(n_networks):
            # Plot spine1 location for each animal
            for k in range(1, animal_ids_hungarian.shape[0]):
                idxb = np.where(animal_ids_hungarian[k]==p)[0]
                if idxb.shape[0]>0:
                    tempx = tracesx_re[k,features+idxb*14]
                    #print ("tempx: ", tempx.shape)
                    tempy = tracesy_re[k, features+idxb*14]
                    locs_array[p,k]= tempx, tempy

        np.save(fname_out, locs_array)
    
    else:
        locs_array = np.load(fname_out)
        
    return locs_array

# 
def cleanup_traces(locs_array, 
                  animal_id,
                  velocity_max,
                  velocity_min,
                  velocity_rel_max,
                  min_seg_len):
    
    data = locs_array[animal_id].copy()
    
    # REMOVE GARBAGE / FAILED PREDICTION
    idx = np.where(data[:,0]==0.)[0]
    #print (" ZERO REMOVAL: ", idx.shape)
    data[idx,0]=np.nan
    data[idx,1]=np.nan

    # DEL SHORT SEGS 
    idx = np.where(np.isnan(data[:,0])==True)[0]
    diffs = idx[1:]-idx[:-1]
    idx2 = np.where(np.logical_and(diffs>1, diffs<min_seg_len))[0]
    #print ("short segs deletion: ", idx2.shape)

    # loop over short segs and remove them
    for id_ in idx2:
        data[idx[id_]:idx[id_+1]]=np.nan

    # VELOCITIES
    if True:

        if False:
            plt.scatter(np.arange(data.shape[0])/25, 
                    data.sum(1), color='black')
            
        #temp = temp1[1:]-temp1[:-1]

        locs_temp = np.sqrt(data[:,0]**2+data[:,1]**2)
        velocity = locs_temp[1:]-locs_temp[:-1]

        # absolute velocity delete
        idx = np.where(np.abs(velocity)>velocity_max)[0]
        data[idx]=np.nan

        # relative velocity delete
        idx = np.where(np.logical_and(np.abs(velocity[1:])>np.abs(velocity[:-1])*velocity_rel_max,
                                     np.abs(velocity[1:])>velocity_min))[0]

        #print ("relative vellcity deletions ", idx.shape)
        data[idx]=np.nan   
        
        idx = np.where(np.logical_and(np.abs(velocity[1:])*velocity_rel_max<np.abs(velocity[:-1]),
                                     np.abs(velocity[1:])>velocity_min))[0]

        #print ("relative vellcity deletions ", idx.shape)
        data[idx]=np.nan   

        

    # DEL SHORT SEGS 
    idx = np.where(np.isnan(data[:,0])==True)[0]
    diffs = idx[1:]-idx[:-1]
    idx2 = np.where(np.logical_and(diffs>1, diffs<min_seg_len))[0]
    #print ("short segs deletion: ", idx2.shape)

    # loop over short segs and remove them
    for id_ in idx2:
        data[idx[id_]:idx[id_+1]]=np.nan
        
    locs_array[animal_id] = data

    return locs_array

def connect_CSUA(locs_array,
                       animal_id,
                       max_jump, 
                       max_jump_intra_animal,     
                       min_nearby_animal_dist):
    
    animal_ids = np.arange(4)
    
    data = locs_array[animal_id].copy()
    #print ("Data: ", data.shape)
    
    
    idx = np.where(np.isnan(data[:,0])==False)[0]
   # print ("interpo: ", idx.shape)
   # print (idx[:10])
    diffs= idx[1:]-idx[:-1]
    idx2 = np.where(diffs>1)[0]
    #print (idx[idx2[:10]])
    
    idxnan = np.where(np.isnan(data[:,0])==True)[0]
    diffsnan = idxnan[1:]-idxnan[:-1]
    idx2nan = np.where(diffsnan>1)[0]
    
    for k in range(1,idx2.shape[0]-1,1):
        start = idx[idx2[k]]
        duration = diffs[idx2[k]]
        end = idxnan[idx2nan[k+1]]+1
        
        dist = np.sqrt((data[start,0]-data[end,0])**2+
                       (data[start,1]-data[end,1])**2)
        
        # CHECK MIN DIST; SET THIS TO CONSERVATIVE VALUE
        if dist <= max_jump:
                
            #print (data[start,0], data[end,0], duration)
            data[start:end,0]= np.linspace(data[start,0], data[end,0], duration)
            data[start:end,1]= np.linspace(data[start,1], data[end,1], duration)
            
        # CHECK MORE BROADLY ENSURE NO OTHER ANIMAL NEARBY
        elif dist <= max_jump_intra_animal:
            for a in animal_ids:
                if a == animal_id:
                    continue
                    #dists_start.append(1E10)
                    #dists_end.append(1E10)
                    
                near_anim = np.sqrt((locs_array[a][start,0]-locs_array[animal_id][start,0])**2+
                                      (locs_array[a][start,1]-locs_array[animal_id][start,1])**2)
                
                if np.min(near_anim)<min_nearby_animal_dist:
                    continue
                    
                near_anim = np.sqrt((locs_array[a][end,0]-locs_array[animal_id][end,0])**2+
                                      (locs_array[a][end,1]-locs_array[animal_id][end,1])**2)
                
                if np.min(near_anim)<min_nearby_animal_dist:
                    continue

            #print (data[start,0], data[end,0], duration)
            data[start:end,0]= np.linspace(data[start,0], data[end,0], duration)
            data[start:end,1]= np.linspace(data[start,1], data[end,1], duration)
            
    locs_array[animal_id] = data
    
    return locs_array



def swap_traces(locs_array, 
                animal_id,
                animal_ids):
    '''  Function that tries to correct obvious swaps
    '''
    
    # GRAB CSUAs FOR ANIMAL AND SEE IF BETTER MATCH WITH OTHER STARTS/ENDS
    # GRAB CSUAs from main animal
    idx = np.where(np.isnan(locs_array[animal_id])==False)[0]
    diffs = idx[1:]-idx[:-1]
    idx_end = np.where(diffs>1)[0]
    
    idxnan = np.where(np.isnan(locs_array[animal_id])==True)[0]
    diffs = idxnan[1:]-idxnan[:-1]
    idx_start = np.where(diffs>1)[0]
    
    CSUA = np.array([idxnan[idx_start],
                    idx[idx_end]]).T
    print ("animal id: ", animal_id, 
           "CSUA start: ", CSUA)
    
    # FIND WHICH CSUAS HAVE LARGE JUMPS FROM NEIGHBOURS
    
    
#     # FIND CSUAs for other animals
#     for a in animal_ids:
#         idx = np.where(np.isnan(locs_array[a])==False)[0]
#         diffs = idx[1:]-idx[:-1]
#         idx2 = np.where(diffs>1)[0]    
    
    
    return locs_array



def cleanup(fname_classification, 
            animal_arrays,
            animal_ids,
            locs_array,
            velocity_max = 200,
            velocity_min = 10.,
            velocity_rel_max = 7.,
            min_seg_len = 5,
            max_jump = 100, # SAFE DISTANCE TO MERGE SPLIT 
            max_jump_intra_animal = 500, # DISTANCE TO MERGE IF NO OTHER ANIMALS "NEARBY"
            min_nearby_animal_dist = 30, # DISTANCE DEFINING "NEARBY"
            separation_dist = 100, # Distance under which labels can no longer change location
           ):

    for animal_id in animal_ids:
        #print (locs_array[animal_id].shape)

        ###################################################
        ################## CLEANUP TRACES #################
        ###################################################
        if True:
            locs_array = cleanup_traces(locs_array,
                              animal_id,
                              velocity_min,
                              velocity_min,
                              velocity_rel_max,
                              min_seg_len)

        ###################################################
        ############# SWAP TRACES - NOT IMPLEMENTED #######
        ###################################################
    #     locs_array = swap_traces(locs_array,
    #                              animal_id,
    #                              animal_ids)

        ###################################################
        ############## CONNECTS CSUAS  ####################
        ###################################################
        if True:
            # REPEAT INTERPOLATION X TIMES
            for k in range(5):
                locs_array = connect_CSUA(locs_array,
                                                animal_id,
                                                max_jump,
                                                max_jump_intra_animal,     
                                                min_nearby_animal_dist)
        ###################################################
        ################# INTERPOLATE TRACES ##############
        ###################################################
        if True:
            # Fix x,y positions for missing data to last value in time
            # TODO: Try to recover missing data by looking at 
            #  raw heatmaps or peaks outputed by DLC
            locs_array = interpolate_traces(locs_array,
                                            animal_id)
        
        
        ###################################################
        ######## STOPS TRACES FROM JUMPING/JITTER #########
        ###################################################
        locs_array = stop_traces_jumps(locs_array, separation_dist)

            

    return locs_array 

def interpolate_traces(locs_array,
                       animal_id):
    
    ''' Replace all np.nans by the last good value
    '''
        
    data = locs_array[animal_id].copy()
    #print ("Data: ", data.shape)
    
    idx = np.where(np.isnan(data)==True)[0]
    for id_ in idx:
        data[id_]=data[id_-1]
    
    # go backwoards in time to ensure beginning of time also has non np.nan vals
    data = data[::-1]
    idx = np.where(np.isnan(data)==True)[0]
    for id_ in idx:
        data[id_]=data[id_-1]

    data = data[::-1]
    locs_array[animal_id] = data
    
    return locs_array

def stop_traces_jumps(locs_array, separation_dist):
    ''' Function that freezes traces once an animal gets near another
    '''
    
    # COMPUTE INTER-ANIMAL DISTANCES AT EVERY POINT:
    
    for a in range(4):
        
        locs_a = locs_array[a]
        
        for aa in range(4):
            if a==aa:
                continue
            locs_aa = locs_array[aa]
            
            dists = np.sqrt((locs_a[:,0]-locs_aa[:,0])**2+
                            (locs_a[:,1]-locs_aa[:,1])**2)
            
            idx = np.where(dists<separation_dist)[0]
            # search forward in time and fix to last value if too close 
            for id_ in idx:
                locs_a[id_]=locs_a[id_-1]

    
    return locs_array

                       
def csv_to_npy(fname_csv):

    ''' Convert csv to a numpy file with only floats
    '''

    import csv
    from tqdm import tqdm
    import numpy as np

    # LOAD .CSV DATA FIRST
    
    fname_out = fname_csv[:-4]+'.npy'
    
    if os.path.exists(fname_out)==False:
        print ("converting sleap .csv to .npy standard")
        data_tracks = []
        with open(fname_csv) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=',')
            ctr=0
            for row in tqdm(csv_reader):
                #print (ctr)
                data_tracks.append([])
                for item in row:
                    if item=='':
                        item = 1E10
                    elif 'track' in item:
                        item = item.replace('track_','')
                    data_tracks[ctr].append(item)
                ctr+=1
        data_tracks = np.array(data_tracks[1:])
        #print (data_tracks[0])

        # CONVERT DATA TO FLOAT32 and replace missing data with Nan
        #print (data_tracks.shape)
        data_tracks = np.float32(data_tracks)
        #print (data_tracks)

        idx = np.where(data_tracks==1E10)
        data_tracks[idx]=np.nan

        print (data_tracks.shape)

        np.save(fname_out,
               data_tracks)

    else:
        data_tracks = np.load(fname_out)
    
    return data_tracks


def classifications_to_standard_output(n_features, 
                                       n_frames,
                                       n_animals,
                                       fname_csv,
                                       fname_classification,
                                       fname_chain_ids,
                                       fname_traces_inferences):
    
    ''' Combine Sleap and CNN classification output to standard format for hungarian algorithm downstream
        input:  CNN output is: classes [n_labels] usually 3-4 x n_frames
                Sleap output converted to track_ids .npy file: [track_id, frame_id, feature1_x, feature1_y,...]
                
                # Note: Sleap output is already converted to:
                #               - chain_ids [n_frames, n_animals]
                #               - traces_inferences [n_frames, n_animals*n_features, 2]
        
        output: [n_frames, n_animals, n_features, 2]
        
    '''
    
    from tqdm import tqdm
    
    # convert the sleap .csv file to an npy containing only floats
    data_tracks = csv_to_npy(fname_csv)
    print ("sleap data tracks;", data_tracks.shape)
    
    # 
    classification = np.load(fname_classification)
    classes = classification['classes']        # classification output
    confidence = classification['confidence']  # likelihood of classifier
    frames_array = classification['frames_array']  # this is the movie frame ids
    vals = classification['vals']  # this is the track ids
    
    tracks_unique = np.int32(np.unique(vals))
    print (" unique tracks: ", tracks_unique)
    
    # start reassigning data;
    traces_inferences_classification = np.zeros((n_frames, n_animals, n_features, 2), 'float32')+np.nan
    print ("traces_inferences_classification: ", traces_inferences_classification.shape)
    
    # loop over all unique tracks and assign them to frames based on classification
    for track_unique in tqdm(tracks_unique):
        idx = np.where(vals==track_unique)[0]
        classes_selected = classes[idx]
        
        # 
        idx2 = np.where(data_tracks[:,0]==track_unique)[0]
        
        for ctr, idx2_selected in enumerate(idx2):
            data_selected = data_tracks[idx2_selected]
            frame_selected = int(data_selected[1])
                
            animal_selected = int(classes_selected[ctr])
            
            features_selected_x = data_selected[2:][::2]
            features_selected_y = data_selected[2:][1::2]

            if frame_selected==12276:
                print (track_unique, ctr, idx2_selected)
                print ("frame_selected :", frame_selected)
                print ("animal_selected :", animal_selected)
                print ("features_selected_x: ", features_selected_x)
                
            traces_inferences_classification[frame_selected, 
                                             animal_selected,:,0]=features_selected_x
            traces_inferences_classification[frame_selected, 
                                             animal_selected,:,1]=features_selected_y
    
    fname_out = fname_traces_inferences[:-4]+"_classification.npy"
    print ("Saved fname: ", fname_out)
    np.save(fname_out,
           traces_inferences_classification)


In [7]:
####################################
######## SET FILENAMES #############
####################################

# Classifier output for CSUA; using 100 x 100 pixel images;
fname_classification = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/classification_output.npz'
                        #/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/classification_output.npz
    
# Notebook 11 - CC based image extraction generates chain_id file 
fname_chain_ids = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis_chain_ids.npy'
chain_ids = np.load(fname_chain_ids)
print ("chain ids: ", chain_ids.shape)

# 
fname_csv = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis.csv'

# CSUA image locations; required for hungarian; may wish to simplify;
root_dir = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/animal_images/'

# Traces post-CC step
fname_traces_inferences = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis_traces_reassembled.npy'
                          #'/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences_0_89988.npz'
traces = np.load(fname_traces_inferences)
print ("traces: ", traces.shape)
    
    

chain ids:  (89987, 4)
traces:  (89987, 56, 2)


In [186]:
# #######################################################################
# ###### COMBINE CNN OUTPUT AND SLEAP OUTPUT + HUNGARIAN ALGORITHM ######
# #######################################################################
# def get_animal_ids_function3(fname_classification,
#                            fname_traces_inferences,
#                            chain_ids,
#                            root_dir,
#                            comments,
#                            n_networks):
    
    
#     #####################################################
#     ############## GET HUNGARYAN ASSIGNMENT #############
#     #####################################################
#     fname_out = os.path.split(fname_classification)[0]+"/animal_arrays.npy"
#     if os.path.exists(fname_out)==False:

#         data = np.load(fname_classification, allow_pickle=True)
#         vals = data['vals']
#         frames = np.hstack(data['frames_array'])
#         print ("# of lableed frames: ", vals.shape)
#         classes = data['classes']

#         reassembled = np.load(fname_traces_inferences)

#         #reassembled = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/pickle/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz')
#         tracesx_re = reassembled[:,:,0]
#         tracesy_re = reassembled[:,:,1]
#         print ("Lenght of tracessx: ", tracesx_re.shape)

#         start = 1
#         end = tracesx_re.shape[0]

#         # INITIALIZE ARRAYS TO BE SAVED
#         animal_ids_hungarian = np.zeros((tracesx_re.shape[0], n_networks), 'float32')
#         animal_arrays = np.zeros((n_networks, tracesx_re.shape[0], 2), 'float32')
#         for n in trange(start,end, 1):
#             #ret, frame = original_vid.read()

#             # animal_ids is the hungarian output;
#             # animal_ids is the CNN output
#             animal_ids, animal_ids2, dirs = get_animal_ids(n,
#                                                         chain_ids,
#                                                         root_dir,
#                                                         vals,
#                                                         classes,
#                                                         n_networks,
#                                                         comments)

#             #print ("animal_ids: ", animal_ids)
#             for k in range(0, 14*n_networks,14):
#                 y = tracesx_re[n,k:k+14]
#                 x = tracesy_re[n,k:k+14]

#                 idx = np.where(x!=0)[0]
#                 x = x[idx]
#                 y = y[idx]
#                 if x.shape[0]==0:
#                     x_ave = None
#                     y_ave = None
#                 else:
#                     x_ave = np.nanmean(x)
#                     y_ave = np.nanmean(y)

#                 if animal_ids[k//14] is None:
#                     continue
#                 animal_arrays[animal_ids[k//14], n] = x_ave, y_ave

#             animal_ids_hungarian[n]=animal_ids 

#         np.save(fname_out, animal_arrays) # these are the mean location of the data
#         np.save(fname_out[:-4]+"_animal_ids_hungarian.npy", animal_ids_hungarian)
    
#     else:
#         animal_arrays = np.load(fname_out)
#         animal_ids_hungarian = np.load(fname_out[:-4]+"_animal_ids_hungarian.npy")


#     #####################################################
#     ####### USE HUNGARIAN OUTPUT TO REMAKE TRACES #######
#     ##################################################### 
#     traces_inferences_classification_hungarian = np.zeros((n_frames, 
#                                                            n_animals, 
#                                                            n_features, 
#                                                            2), 'float32')+np.nan
#     #
#     print ("Combining hungarian and sleap output")
    
#     # combine hungarian output and slp output
#     traces_inferences = np.load(fname_traces_inferences)
#     tracesx_re = traces_inferences[:,:,0]
#     tracesy_re = traces_inferences[:,:,1]
#     for frame in trange(1,n_frames,1):
#         animals_selected = animal_ids_hungarian[frame]
#         #print ("animals_selected: ", animals_selected)

#         for k in range(n_networks):
#             if np.isnan(animals_selected[k]):
#                 continue
            
#             animal_selected = int(animals_selected[k])
                
#             featuresx = tracesx_re[frame,k*n_features:(k+1)*n_features]
#             featuresy = tracesy_re[frame,k*n_features:(k+1)*n_features]

#             # 
#             traces_inferences_classification_hungarian[frame,
#                           animal_selected,:,0]=featuresx

#             traces_inferences_classification_hungarian[frame,
#                           animal_selected,:,1]=featuresy

#     np.save(fname_traces_inferences[:-4]+"_hungarian.npy", traces_inferences_classification_hungarian)
            
#     print ("DONE making animal_arrays: ", animal_arrays.shape)
    
#     return animal_arrays, animal_ids_hungarian, traces_inferences_classification_hungarian



# comments=False
# n_animals = 4
# n_features = 14

# (animal_ids, animal_ids_hungarian, traces_inferences_classification_hungarian) = get_animal_ids_function2(
#                                                                                            fname_classification,
#                                                                                            fname_traces_inferences,
#                                                                                            chain_ids,
#                                                                                            root_dir,
#                                                                                            comments,
#                                                                                            n_animals,
#                                                                                            n_features)

# print (animal_ids.shape, animal_ids[:,10],
#       animal_ids_hungarian.shape)
# print ("DONE computing hungarian ID assignments")

In [205]:
#################################################
########## UPDATED UNGARIAN ALGORITHM ###########
#################################################

def make_movie(fname_video,
               traces
              ):
    
    #colors_4= ['orange','green', 'blue', 'red', 'cyan']
    colors_4= ['blue','red', 'cyan', 'green', 'yellow']

    video_name = fname_video#'/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi'
    #video_name = root_dir + '/' + os.path.split(root_dir)[1]+'.avi'
    
    print ("video name: ", video_name)
    original_vid = cv2.VideoCapture(video_name)
    font = cv2.FONT_HERSHEY_PLAIN

    # video sizes
    size_vid = np.array([1280,1024])
    scale = 1
    dot_size = 8//scale

    n_features =14

    #out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'

    start = 8*60*25
    end = start+1000
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

    # Initialize video out
    fname_out = video_name[:-4]+"_locs_added_"+str(start)+"_"+str(end)+".mp4"
    fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
    video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0]//scale,size_vid[1]//scale), True)

    for n in trange(start,end, 1):
        ret, frame = original_vid.read()
        cv2.putText(frame, str(n), (50, 100), font, 5, (255, 255, 0), 5)
        frame = frame[::scale, ::scale]

        # loop over the info


        #for k in range(locs.shape[0]):
        for k in range(n_features*n_networks):

            #traces_inferences[frame][k*14:(k+1)*14])
            y = traces_inferences[n,k,0]
            x = traces_inferences[n,k,1]

            #y = locs[k,n,0]
            #x = locs[k,n,1]

            if np.isnan(x) or np.isnan(y) or np.isnan(animal_ids_hungarian[n][k//14]):
                continue
            else:
                x=int(x)//scale
                y=int(y)//scale

                # select color index
                p = int(animal_ids_hungarian[n][k//14])

                frame[x-dot_size:x+dot_size,y-dot_size:y+dot_size]= (np.float32(
                    matplotlib.colors.to_rgb(colors_4[p]))*255.).astype('uint8')
                #print (colors_4[k])
                #frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
                #    matplotlib.colors.to_rgb(colors_4[z//14]))*255.).astype('uint8')

        #print ("")
        video_out.write(frame)

        #print ("")

    video_out.release()
    original_vid.release()
    #cv2.destroyAllWindows()

        
def get_classes(track_ids,
                 track_id, 
                 classes,
                 confidence,
                 frames_array,
                 plotting):
    
    ''' Visualize the distribution of confidence in a track
    
        Input:
        - track_ids
        - track_id  <- selected track id
        - classes
        - confidence
    
    '''
    clrs = ['red','blue','cyan','green']
    n_animals = 4
    fps= 25
    
    idx = np.where(track_ids==track_id)[0]

    classes_pred = classes[idx]
    confidence_pred = confidence[idx,classes_pred]
    frames_pred = frames_array[idx]
    length_frames = int(frames_pred[-1]-frames_pred[0])+1
        
    if plotting:
        gs = gridspec.GridSpec(2,4)
        plt.suptitle("Distribution of confidence across all IDs for track "+str(track_id)+
                    " with # frames: " +str(idx.shape[0])+
                    " ("+str(frames_pred[0])+ " to "+str(frames_pred[-1])+")")
    
    #times = np.zeros((idx.shape[0],n_animals),'int32')
    times = np.zeros((length_frames,n_animals),'int32')
    
    for p in range(n_animals):

        idx1 = np.where(classes_pred==p)[0]
        times[idx1,p]+=1
        temp = confidence_pred[idx1]
        y = np.histogram(temp, np.arange(0,1.1,0.1))

        if plotting:
            ax = plt.subplot(gs[p//4, p%4])
            ax.set_title("ID: "+str(p))
            ax.plot(y[1][:-1],y[0], c=clrs[p])

            ax2 = plt.subplot(gs[1, :])
            ax2.scatter(frames_pred[idx1]/fps, idx1*0+0.01*p, c=clrs[p])

    
    # show rolling curves for cumulative ids;
    window = fps
    t=np.arange(frames_pred[0], frames_pred[-1]+1,1)/window
    for p in range(n_animals):
        temp = np.convolve(times[:,p], np.ones(window)/window, mode='same')
        
        if plotting:
            ax2.plot(t, temp, clrs[p])
            ax2.set_title("Predicted animal vs. time w. rolling window average (1sec)",fontsize=20)
            ax2.set_xlabel("Real Time (sec)", fontsize=20)
            ax2.set_xlim(t[0],t[-1])
            
    if plotting:
        plt.show()

    return classes_pred, confidence_pred         
        
        
def post_CNN_hungarian(root_dir_cnn,
                       fname_sleap_output,
                       fname_classification,
                       fname_chain_ids,
                       fname_traces_inferences,
                       n_animals,
                       n_features,
                       track_id):
    
    '''  Function which implements a time-based hungarian constraint on the CNN outputs
    
        Input: classification data
        - classes [n_images]
        - confidence [n_images, n_animals]
        
        Output: hungarian corrected classification data
        - classes [n_images]
    
    '''
    
    # load sleap output
    sleap = np.load(fname_sleap_output)
    print ("sleap output: ", sleap.shape, sleap[0])
    
    # load CNN classification output; 
    data = np.load(fname_classification)
    classes = data['classes']              #predicted class for each image
    confidence = data['confidence']
    frames_array = data['frames_array']
    track_ids = data['vals'].astype('int32')    # DO NOT change order, 
    print (track_ids.shape)

    # loop over all tracks and get a sense of purity
    
    if track_id is 'all':
        ids = np.arange(track_ids.shape[0])
    else:
        ids = [track_id]
    for k in ids:
        plotting = True
        classes_pred, confidence_pred = get_classes(track_ids,
                                                    k,
                                                    classes,
                                                    confidence,
                                                    frames_array,
                                                    plotting)
        
   
#
n_animals = 4
n_features = 14

# 
root_dir_cnn = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/animal_images/'

# 
fname_sleap =             '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis.npy'
fname_sleap_traces =      '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis_traces_reassembled.npy'
fname_classification =    '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/classification_output.npz'

# need to make this file which contains the sleap tracks -> traces shape -> with IDs corrected by CNN 
# this is required for making movies/analysis downstream
fname_chain_ids =         '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis_chain_ids.npy'
  

track_id = 120
#
post_CNN_hungarian(root_dir_cnn,
                   fname_sleap_output,
                   fname_classification,
                   fname_chain_ids,
                   fname_traces_inferences,
                   n_animals,
                   n_features,
                   track_id)





    
    
    

<>:184: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:184: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-205-f6033abfe2e2>:184: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if track_id is 'all':


sleap output:  (307652, 30) [  0.        0.      310.35138 572.4292        nan       nan 328.01074
 592.1255        nan       nan 332.36166 620.2585  322.48322 642.2069
 344.07208 678.09973 352.18756 696.1015  374.36697 704.43665 432.44778
 724.13495 458.30075 742.2128        nan       nan       nan       nan
       nan       nan]
(307652,)


<ipython-input-205-f6033abfe2e2>:128: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax2 = plt.subplot(gs[1, :])
<ipython-input-205-f6033abfe2e2>:128: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  ax2 = plt.subplot(gs[1, :])
<ipython-input-205-f6033abfe2e2>:128: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will 

In [180]:
#######################################################
###### COMBINE SLEAP WITH CNN INTO TRACES FILE ########
#######################################################

# 
def make_sleap_traces_classification(fname_chain_ids,
                                     fname_sleap_traces,
                                     fname_classification):
        
    '''  Function to make traces file which contains sleap output with CNN corrected ID in a simple format
    
        Input:
        - sleap_traces file converted to .npy format : [n_images, track_id+frame_id+features*2]; 
        - chain_ids [n_frames, n_animals] 
        - classification data: classes [n-images], confidence [n_images, n_animals], track_ids [n_tracks]
        
        Output:
        - sleap_traces_classification file : [n_frames, n_animals, n_features, 2]
    
    '''
    from tqdm import tqdm
    
    n_animals = 4
    n_features = 14
    
    # load sleap traces
    sleap_traces = np.load(fname_sleap_traces)
    sleap_traces = sleap_traces.reshape(sleap_traces.shape[0],
                                       n_animals,
                                       n_features,
                                       2)
    # 
    print ("sleap traces: ", sleap_traces.shape)
    
    # make final traces 
    sleap_traces_classification = np.zeros(sleap_traces.shape, 'float32')+np.nan
    print ("sleap sleap_traces_classification: ", sleap_traces_classification.shape)

    # load chain ids which show conversion between sleap output and traces
    chain_ids = np.load(fname_chain_ids)
    print ("chain ids: ", chain_ids.shape)
    
    # 
    chain_ids_corrected = np.zeros(chain_ids.shape, 'float32')+np.nan
    
    # load CNN classification output
    data = np.load(fname_classification)
    classes = data['classes']              #predicted class for each image
    confidence = data['confidence']
    frames_array = data['frames_array']
    track_ids = data['vals'].astype('int32')    # DO NOT change order, 
    print ("# of images classified by sleap: ", track_ids.shape)
    
    # 
    idx_tracks = np.int32(np.unique(track_ids))
    for id_ in tqdm(idx_tracks):
    #for id_ in [0,1732]:
        
        #print (" track id: ", id_)
        # get index location to the x,y data in the 
        frames_, animals_original_ = np.where(chain_ids==id_)
        #print ("frames_ ", frames_ )
        #print ("original animals ", animals_original_ )
        
        # find location in sleap file
        idx2 = np.where(track_ids==id_)[0]
        #print ("track ", id_, " locations: ", idx2)
        
        # find class ID in sleap file
        animals_predicted_ = classes[idx2]
        #print ("predicted animals_: ", animals_predicted_)
        
        # find frames THIS IS REDUNDANT
        # frames_ = np.int32(frames_array[idx2])
        # print ("frames: ", frames_)
        
        # assign data into the
        for f in range(frames_.shape[0]):
            #for a in range(animals_predicted_.shape[0]):
            sleap_traces_classification[frames_[f], 
                        animals_predicted_[f]]= sleap_traces[frames_[f],
                                                        animals_original_[f]]
        
            chain_ids_corrected[frames_[f],animals_predicted_[f]] = id_
    
    # save corrected traces
    fname_sleap_traces_classification = fname_sleap_traces[:-4]+"_classification.npy"
    np.save(fname_sleap_traces_classification, sleap_traces_classification)

    # save corrected chain_ids file
    fname_chain_ids_corrected = fname_chain_ids[:-4]+"_classification.npy"
    np.save(fname_chain_ids_corrected, chain_ids_corrected)
    
    
    
    return sleap_traces_classification


# 
# fname_sleap =             '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis.npy'
fname_chain_ids =         '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis_chain_ids.npy'
fname_sleap_traces =      '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis_traces_reassembled.npy'
fname_classification =    '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/classification_output.npz'

# need to make this file which contains the sleap tracks -> traces shape -> with IDs corrected by CNN 
# this is required for making movies/analysis downstream

# 
sleap_traces_classification = make_sleap_traces_classification(fname_chain_ids,
                                                             fname_sleap_traces,
                                                             fname_classification)

# 






  2%|▏         | 50/2187 [00:00<00:04, 491.50it/s]

sleap traces:  (89987, 4, 14, 2)
sleap sleap_traces_classification:  (89987, 4, 14, 2)
chain ids:  (89987, 4)
# of images classified by sleap:  (307652,)


100%|██████████| 2187/2187 [00:04<00:00, 464.71it/s]


In [207]:
# show videos
    
# use the original post-classification
fname_traces = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis_traces_reassembled_classification.npy'
fname_chain_ids = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis_chain_ids_classification.npy'
video_name = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi'

start = 2598
end = 2874

#
make_video(video_name,
           fname_chain_ids,
           fname_traces,
           start,
           end)

100%|██████████| 276/276 [00:06<00:00, 42.78it/s]


In [22]:
data = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis.npy')
print (data.shape)

chain_ids = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis_chain_ids.npy')

print (chain_ids, chain_ids.shape)

(307652, 30)
[[0.000e+00 1.000e+00       nan       nan]
 [0.000e+00 1.000e+00       nan       nan]
 [0.000e+00 1.000e+00       nan       nan]
 ...
 [2.181e+03 2.184e+03 2.187e+03 2.188e+03]
 [2.181e+03 2.184e+03 2.187e+03 2.188e+03]
 [2.181e+03 2.184e+03 2.187e+03 2.188e+03]] (89987, 4)


In [38]:
# CHECK TRACK IDS
track_id = 285

fname = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/animal_images/'+str(track_id)+'/predictions.npz'
data = np.load(fname)
predictions = data['predictions']
print (np.unique(predictions, return_counts=True))
confidence = data['confidence']
frame_ids = data['frame_ids']
print (confidence)
print (frame_ids)

(array([0, 1, 2, 3]), array([ 272, 1804, 1021,  265]))
[[2.2059169e-06 5.6868776e-05 4.8914370e-01 5.1079720e-01]
 [8.6293346e-04 9.9825829e-01 6.8832225e-07 8.7806600e-04]
 [7.1768379e-03 9.8313981e-01 9.0713743e-03 6.1205594e-04]
 ...
 [4.4628687e-05 9.9926239e-01 3.3892933e-04 3.5404877e-04]
 [8.1040021e-03 7.4503267e-01 2.4213201e-01 4.7313678e-03]
 [7.5283046e-06 9.9440986e-01 3.4747529e-04 5.2351169e-03]]
[ 9887.  9888.  9889. ... 13265. 13268. 13269.]


(89987, 4, 14, 2)


[[0.000e+00 1.000e+00       nan       nan]
 [0.000e+00 1.000e+00       nan       nan]
 [0.000e+00 1.000e+00       nan       nan]
 ...
 [2.181e+03 2.184e+03 2.187e+03 2.188e+03]
 [2.181e+03 2.184e+03 2.187e+03 2.188e+03]
 [2.181e+03 2.184e+03 2.187e+03 2.188e+03]]


In [12]:
######################################################################
######## MERGE CLASSIFICAITON AND TRACES_INFERENCES OUTPUT ###########
######################################################################
def get_animal_ids_confidence(frame,
                              chain_ids,
                              root_dir,
                              vals,
                              classes,
                              n_animals,
                              comments=False):

    # 
    dirs = chain_ids[frame]

    if comments:
        print ("DIRS: ", dirs)
    
    animal_ids = []
    animal_ids2 = []
    threshold_purity1 = 0.5
    threshold_purity2 = 0.7

    # HUNGARIAN ALGORITHM BASED SEARCH FOR MOST CORRECT IDS
    best_ids_cts = np.zeros((5,5),'float32')
    animals3 = []
    for ctr_dir, dir_ in enumerate(dirs):
        #print ("2nd Loop: ", dir_)
        if np.isnan(dir_)==False:
            
            # find the classifier values for that segment
            idx = np.where(vals==dir_)[0]

            # find predicted id;
            # First find 
            #fnames = np.sort(glob.glob(root_dir+str(int(dir_))+'/*.npy',recursive = False))
            fnames = np.sort(glob.glob(root_dir+str(int(dir_))+'/*.npz',recursive = False))

            ctr_chain = 0
            for fname in fnames:
                if ('frame_'+str(frame).zfill(7)) in fname:
                     break 
                ctr_chain+=1
        
            animal_id3 = classes[idx][ctr_chain]
            animals3.append(animal_id3)
            
            idx5 = np.where(classes[idx]<1E10)[0]
            classes_local = classes[idx[idx5]]
            
            classes_unique = np.unique(classes_local, return_counts=True)
            #print ("best id: ", best_id)
            
            best_ids_cts[ctr_dir, classes_unique[0]]=classes_unique[1]/idx.shape[0]
        else:
            animals3.append(None)
    
    # FIRST LOOP TO CHECK ANY GREAT MATCHES AND ZERO OUT EVERYTHING ELSE
    for k1 in range(n_animals):
        temp = best_ids_cts[k1]
        max_val = np.max(temp)
        if max_val>threshold_purity2:
            idx8 = np.argmax(temp)
            best_ids_cts[:,idx8]=0
            best_ids_cts[k1,idx8]=max_val
            
    if comments:
        print (best_ids_cts)
    
    # RENORMALIZE PROBABILITIES AFTER ZEROING OUT
    for k1 in range(n_animals):
        sum_ = best_ids_cts[k1].sum()
        if sum_>0:
            best_ids_cts[k1]*= 1./sum_
    if comments:
        print (best_ids_cts)    
        
    # FINAL LOOP TO GRAB WHATEVER IS LEFT
    for k1 in range(n_animals):
        max_val = np.max(best_ids_cts[k1])
        if max_val==0: # or max_val<threshold_purity1:
            animal_ids.append(None)
            animal_ids2.append(animals3[k1])
            continue
        else:
            idx9 = np.argmax(best_ids_cts[k1])
            #print ("idx9: ", idx9, classes_unique[0])
            animal_ids.append(idx9)
            animal_ids2.append(animals3[k1])
            
            best_ids_cts[:,idx9]= 0
            # must zero out all the others
            
    if comments:
        print (best_ids_cts)   
        
    return animal_ids, animal_ids2, dirs


traces_inferences = np.load(fname_traces_inferences)
classification = np.load(fname_classification)
classes = classification['classes']
confidence = classification['confidence']
frames_array = classification['frames_array']
print ('frames_array: ', frames_array.shape)
print ("classes: ", classes.shape, classes[:100])
vals = classification['vals']
print ("vals ", vals[:100])

#
n_animals=4

#
frame = 0

#
print ("chain ids: ", chain_ids.shape, 
       chain_ids[frame])

#
animal_ids, animal_ids2, dirs = get_animal_ids_confidence(frame,
                                                          chain_ids,
                                                          root_dir,
                                                          vals,
                                                          classes,
                                                          n_animals,
                                                          comments=False)
    

    




frames_array:  (307652,)
classes:  (307652,) [0 0 0 0 2 0 0 0 0 2 2 0 0 0 2 0 3 3 0 3 0 0 2 2 0 0 0 2 0 0 3 3 0 2 2 2 1
 0 2 0 0 0 0 0 0 3 0 0 2 2 0 0 0 0 3 0 1 0 0 0 0 1 1 0 0 0 2 0 0 2 0 0 0 0
 0 2 0 0 0 0 0 0 0 0 2 0 3 0 0 2 3 0 0 0 0 0 0 2 0 1]
vals  [831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831.
 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831.
 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831.
 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831.
 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831.
 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831.
 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831.
 831. 831.]
chain ids:  (89987, 4) [ 0.  1. nan nan]


In [86]:
###################################
###################################
###################################
traces_inferences = np.load(fname_traces_inferences)
classification = np.load(fname_classification)
classes = classification['classes']
confidence = classification['confidence']
frames_array = classification['frames_array']
print ('frames_array: ', frames_array.shape)
print ("classes: ", classes.shape, classes[:100])
vals = classification['vals']
print ("vals ", vals[:100])
frame = 2555

#
# print ("classes: ", classes.shape, classes[frame])
# print ('vals: ', vals.shape, vals[frame])
# for k in range(4):
#     print (traces_inferences[frame, k*14:(k+1)*14][:3])
#     print ('')

print ('chain ids: ', chain_ids.shape, chain_ids[1])
print ("traces: ", traces_inferences.shape)
print ("classification animal ids: ", animal_ids.shape, animal_ids[0,1])
print ("classification animal_ids_hungarian: ", animal_ids_hungarian.shape, animal_ids_hungarian[1])


frames_array:  (307652,)
classes:  (307652,) [0 0 0 0 2 0 0 0 0 2 2 0 0 0 2 0 3 3 0 3 0 0 2 2 0 0 0 2 0 0 3 3 0 2 2 2 1
 0 2 0 0 0 0 0 0 3 0 0 2 2 0 0 0 0 3 0 1 0 0 0 0 1 1 0 0 0 2 0 0 2 0 0 0 0
 0 2 0 0 0 0 0 0 0 0 2 0 3 0 0 2 3 0 0 0 0 0 0 2 0 1]
vals  [831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831.
 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831.
 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831.
 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831.
 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831.
 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831.
 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831. 831.
 831. 831.]
chain ids:  (89987, 4) [ 0.  1. nan nan]
traces:  (89987, 56, 2)
classification animal ids:  (4, 89987, 2) [669.44037 373.67206]
classification animal_ids_hungarian:  (89987, 4) [ 0.  1. nan nan]


In [57]:
frame = 22256
for k in range(1):
    print (traces_inferences[frame][k*14:(k+1)*14])
    #print (animal_ids[k,frame][::-1])
    #print (animal_ids[int(animal_ids_hungarian[frame][k]),frame][::-1])
    print ("animal ids hungarian: ", animal_ids_hungarian[frame])
    print('')


[[1006.34607  618.2427 ]
 [1004.34564  658.5099 ]
 [1040.4918   644.3095 ]
 [1006.505    692.188  ]
 [1056.4115   674.3973 ]
 [1034.6678   694.457  ]
 [1024.6968   724.305  ]
 [1008.36365  744.2123 ]
 [ 984.416    750.198  ]
 [ 958.69275  746.26917]
 [ 934.603    748.47144]
 [ 906.5242   754.2024 ]
 [ 878.5866   748.6434 ]
 [       nan        nan]]
[988.05005 707.56964]
animal ids hungarian:  [1. 0. 2. 3.]



In [74]:
############################################
############ MAKE VIDEO BASES ON TRACES ####
############################################
#          pup1     pup2    female  male
# go through first videos
from tqdm import trange

colors_4= ['orange','green', 'blue', 'red', 'cyan']
colors_4= ['blue','red', 'cyan', 'green', 'yellow']

video_name = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi'
#video_name = root_dir + '/' + os.path.split(root_dir)[1]+'.avi'
print ("video name: ", video_name)
original_vid = cv2.VideoCapture(video_name)
font = cv2.FONT_HERSHEY_PLAIN

# video sizes
size_vid = np.array([1280,1024])
scale = 1
dot_size = 8//scale

n_features =14

#out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'

start = 8*60*25
end = start+1000
original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

# Initialize video out
fname_out = video_name[:-4]+"_locs_added_"+str(start)+"_"+str(end)+".mp4"
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0]//scale,size_vid[1]//scale), True)

for n in trange(start,end, 1):
    ret, frame = original_vid.read()
    cv2.putText(frame, str(n), (50, 100), font, 5, (255, 255, 0), 5)
    frame = frame[::scale, ::scale]
    
    # loop over the info
    
    
    #for k in range(locs.shape[0]):
    for k in range(n_features*n_networks):
    
        #traces_inferences[frame][k*14:(k+1)*14])
        y = traces_inferences[n,k,0]
        x = traces_inferences[n,k,1]
        
        #y = locs[k,n,0]
        #x = locs[k,n,1]
        
        if np.isnan(x) or np.isnan(y) or np.isnan(animal_ids_hungarian[n][k//14]):
            continue
        else:
            x=int(x)//scale
            y=int(y)//scale
            
            # select color index
            p = int(animal_ids_hungarian[n][k//14])

            frame[x-dot_size:x+dot_size,y-dot_size:y+dot_size]= (np.float32(
                matplotlib.colors.to_rgb(colors_4[p]))*255.).astype('uint8')
            #print (colors_4[k])
            #frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
            #    matplotlib.colors.to_rgb(colors_4[z//14]))*255.).astype('uint8')
                
    #print ("")
    video_out.write(frame)

    #print ("")

video_out.release()
original_vid.release()
#cv2.destroyAllWindows()


video name:  /media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi


100%|██████████| 1000/1000 [00:24<00:00, 40.83it/s]


In [16]:
# data = np.load('/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi.predictions.analysis_chain_ids.npy')
# print (data.shape)
# print (data[:10])

(89987, 4)
[[ 0.  1. nan nan]
 [ 0.  1. nan nan]
 [ 0.  1. nan nan]
 [ 0.  1. nan nan]
 [ 0.  1.  2. nan]
 [ 0.  1. nan nan]
 [ 0.  1. nan nan]
 [ 0.  1.  2. nan]
 [ 0.  1. nan nan]
 [ 0.  1.  2. nan]]


In [18]:
##############################################################
####### COMPUTE SINGLE FEATURE TRACES POST HUNGARIAN #########
##############################################################
features = np.array(6, 'int32') # spine_1
#ids = np.arange(5)
#ids=[0]

locs_array = get_single_feature_traces(fname_classification,
                                      fname_traces_inferences,
                                      animal_ids_hungarian,
                                      n_networks,
                                      features)

print ("DONE with single feature post huganrian", locs_array.shape)

DONE with single feature post huganrian (4, 89987, 2)


In [19]:
#################################################################
###### CLEANUP: Del short segs; connect safe segs ###############
#################################################################
clrs = ['blue','red','yellow','green','yellow']

animal_arrays = np.load(os.path.split(fname_classification)[0]+'/animal_arrays.npy')
animal_arrays = locs_array.copy()

animal_ids = np.arange(n_networks)
locs_array_clean = cleanup(fname_classification, 
                            animal_arrays,
                            animal_ids,
                            locs_array)
            
np.save(os.path.split(fname_classification)[0]+'/locs_array_clean.npy', 
                       locs_array_clean)

print ("DONE Post-processing traces - GO TO Notebook 15_behaviour_analysis")

DONE Post-processing traces - GO TO Notebook 15_behaviour_analysis


In [33]:
fname_classification = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/classification_output.npz'
data = np.load(fname_classification)
confidence = data['confidence']
print (confidence.shape)
print (confidence)

classes = data['classes']
print (classes)





(307652,)
[0.7935309  0.9942002  0.86842144 ... 0.80841905 0.72285736 0.9090116 ]
[0 0 0 ... 2 0 2]
